In [ ]:
import pandas as pd   
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error, r2_score, mean_squared_error, accuracy_score, precision_score, recall_score, f1_score

from xgboost import XGBClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
import statsmodels.api as sm
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning:

pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.



In [ ]:
df_chfusd = pd.read_csv('/content/CHF-USD Exchange Rate Clean.csv')
#df_trends = pd.read_excel('/content/World_GT_Full.xlsx')

In [ ]:
#df_trends['Date'] = pd.to_datetime(df_trends.iloc[:, 0], infer_datetime_format=True)
df_chfusd['Date'] = pd.to_datetime(df_chfusd.iloc[:, 0], infer_datetime_format=True)

In [ ]:
df_chfusd['CHF/USD Log Difference'] = np.log(df_chfusd['Adj Close']) - (np.log(df_chfusd['Adj Close']).shift(1))
df_chfusd['CHF/USD Log Difference']
df_chfusd = df_chfusd.dropna()

In [ ]:
#ts_df = df_chfusd.merge(df_trends, how = 'inner', on = ['Date'])
ts_df = df_chfusd
ts_df

,Date,Adj Close,Log_Difference,CHF/USD Log Difference
1,2003-09-18,0.721605,-0.003686,-0.003686
2,2003-09-19,0.731582,0.013731,0.013731
3,2003-09-22,0.739208,0.010370,0.010370
4,2003-09-23,0.735998,-0.004352,-0.004352
5,2003-09-24,0.739645,0.004943,0.004943
...,...,...,...,...
4690,2021-09-08,1.087607,-0.004983,-0.004983
4691,2021-09-09,1.084787,-0.002596,-0.002596
4692,2021-09-10,1.091096,0.005799,0.005799
4693,2021-09-13,1.089159,-0.001777,-0.001777


In [ ]:
ts_df = ts_df.drop(['Log_Difference','Adj Close', 'Log_Difference'], axis = 1)
ts_df = ts_df.set_index(ts_df.iloc[:,0])
ts_df = ts_df.drop(['Date'], axis = 1)
ts_df

,CHF/USD Log Difference
Date,
2003-09-18,-0.003686
2003-09-19,0.013731
2003-09-22,0.010370
2003-09-23,-0.004352
2003-09-24,0.004943
...,...
2021-09-08,-0.004983
2021-09-09,-0.002596
2021-09-10,0.005799


In [ ]:
#ts_df.iloc[:,1:155] = ts_df.iloc[:,1:155].diff()

In [ ]:
#ts_df = ts_df.iloc[1:,:]

In [ ]:
ts_df.head()

,CHF/USD Log Difference
Date,
2003-09-18,-0.003686
2003-09-19,0.013731
2003-09-22,0.010370
2003-09-23,-0.004352
2003-09-24,0.004943


In [ ]:
# Creating a copy of the initial datagrame to make various transformations 
data = pd.DataFrame(ts_df.copy())

In [ ]:
window_size = 2
data['CHF/USD_RealVol'] = data['CHF/USD Log Difference'].rolling(window_size).std()*(252**0.5)

In [ ]:
data['CHF/USD_RealVol'].quantile(0.75) #75th Percentile

0.10154467619940855

In [ ]:
# Adding the lag of the target variable from 6 steps back up to 24

for column in data:

  for i in range(1, 14):
      data[f"{column}_lag_{i}"] = data[column].shift(i)

In [ ]:
data['HighVolMoment_CHF/USD'] = [1 if i > 0.10 else 0 for i in data['CHF/USD_RealVol']]

In [ ]:
data

,CHF/USD Log Difference,CHF/USD_RealVol,CHF/USD Log Difference_lag_1,CHF/USD Log Difference_lag_2,CHF/USD Log Difference_lag_3,CHF/USD Log Difference_lag_4,CHF/USD Log Difference_lag_5,CHF/USD Log Difference_lag_6,CHF/USD Log Difference_lag_7,CHF/USD Log Difference_lag_8,CHF/USD Log Difference_lag_9,CHF/USD Log Difference_lag_10,CHF/USD Log Difference_lag_11,CHF/USD Log Difference_lag_12,CHF/USD Log Difference_lag_13,CHF/USD_RealVol_lag_1,CHF/USD_RealVol_lag_2,CHF/USD_RealVol_lag_3,CHF/USD_RealVol_lag_4,CHF/USD_RealVol_lag_5,CHF/USD_RealVol_lag_6,CHF/USD_RealVol_lag_7,CHF/USD_RealVol_lag_8,CHF/USD_RealVol_lag_9,CHF/USD_RealVol_lag_10,CHF/USD_RealVol_lag_11,CHF/USD_RealVol_lag_12,CHF/USD_RealVol_lag_13,HighVolMoment_CHF/USD
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2003-09-18,-0.003686,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2003-09-19,0.013731,0.195514,-0.003686,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2003-09-22,0.010370,0.037731,0.013731,-0.003686,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.195514,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2003-09-23,-0.004352,0.165254,0.010370,0.013731,-0.003686,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.037731,0.195514,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2003-09-24,0.004943,0.104335,-0.004352,0.010370,0.013731,-0.003686,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.165254,0.037731,0.195514,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-09-08,-0.004983,0.043176,-0.001137,0.000710,0.001123,-0.000237,0.002051,-0.006541,0.007370,-0.004695,-0.000591,-0.000877,0.005270,0.001961,-0.001481,0.020731,0.004629,0.015264,0.025689,0.096450,0.156145,0.135427,0.046071,0.003212,0.068997,0.037140,0.038636,0.017805,0
2021-09-09,-0.002596,0.026791,-0.004983,-0.001137,0.000710,0.001123,-0.000237,0.002051,-0.006541,0.007370,-0.004695,-0.000591,-0.000877,0.005270,0.001961,0.043176,0.020731,0.004629,0.015264,0.025689,0.096450,0.156145,0.135427,0.046071,0.003212,0.068997,0.037140,0.038636,0
2021-09-10,0.005799,0.094237,-0.002596,-0.004983,-0.001137,0.000710,0.001123,-0.000237,0.002051,-0.006541,0.007370,-0.004695,-0.000591,-0.000877,0.005270,0.026791,0.043176,0.020731,0.004629,0.015264,0.025689,0.096450,0.156145,0.135427,0.046071,0.003212,0.068997,0.037140,0


In [ ]:
data = data.iloc[14:,:]

In [ ]:
#fin_data = [data.iloc[:,0:1], data.iloc[:,156:]]
#final_data = pd.concat(fin_data, axis=1)

In [ ]:
final_data = data.copy()
final_data

,CHF/USD Log Difference,CHF/USD_RealVol,CHF/USD Log Difference_lag_1,CHF/USD Log Difference_lag_2,CHF/USD Log Difference_lag_3,CHF/USD Log Difference_lag_4,CHF/USD Log Difference_lag_5,CHF/USD Log Difference_lag_6,CHF/USD Log Difference_lag_7,CHF/USD Log Difference_lag_8,CHF/USD Log Difference_lag_9,CHF/USD Log Difference_lag_10,CHF/USD Log Difference_lag_11,CHF/USD Log Difference_lag_12,CHF/USD Log Difference_lag_13,CHF/USD_RealVol_lag_1,CHF/USD_RealVol_lag_2,CHF/USD_RealVol_lag_3,CHF/USD_RealVol_lag_4,CHF/USD_RealVol_lag_5,CHF/USD_RealVol_lag_6,CHF/USD_RealVol_lag_7,CHF/USD_RealVol_lag_8,CHF/USD_RealVol_lag_9,CHF/USD_RealVol_lag_10,CHF/USD_RealVol_lag_11,CHF/USD_RealVol_lag_12,CHF/USD_RealVol_lag_13,HighVolMoment_CHF/USD
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2003-10-08,0.002515,0.016099,0.003949,0.013026,-0.014013,-0.002203,0.001748,0.004619,0.014178,0.001638,0.005414,0.004943,-0.004352,0.010370,0.013731,0.101894,0.303516,0.132569,0.044348,0.032229,0.107295,0.140761,0.042383,0.005283,0.104335,0.165254,0.037731,0.195514,0
2003-10-09,-0.005326,0.088012,0.002515,0.003949,0.013026,-0.014013,-0.002203,0.001748,0.004619,0.014178,0.001638,0.005414,0.004943,-0.004352,0.010370,0.016099,0.101894,0.303516,0.132569,0.044348,0.032229,0.107295,0.140761,0.042383,0.005283,0.104335,0.165254,0.037731,0
2003-10-10,0.003573,0.099890,-0.005326,0.002515,0.003949,0.013026,-0.014013,-0.002203,0.001748,0.004619,0.014178,0.001638,0.005414,0.004943,-0.004352,0.088012,0.016099,0.101894,0.303516,0.132569,0.044348,0.032229,0.107295,0.140761,0.042383,0.005283,0.104335,0.165254,0
2003-10-13,-0.007285,0.121876,0.003573,-0.005326,0.002515,0.003949,0.013026,-0.014013,-0.002203,0.001748,0.004619,0.014178,0.001638,0.005414,0.004943,0.099890,0.088012,0.016099,0.101894,0.303516,0.132569,0.044348,0.032229,0.107295,0.140761,0.042383,0.005283,0.104335,1
2003-10-14,0.004850,0.136204,-0.007285,0.003573,-0.005326,0.002515,0.003949,0.013026,-0.014013,-0.002203,0.001748,0.004619,0.014178,0.001638,0.005414,0.121876,0.099890,0.088012,0.016099,0.101894,0.303516,0.132569,0.044348,0.032229,0.107295,0.140761,0.042383,0.005283,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-09-08,-0.004983,0.043176,-0.001137,0.000710,0.001123,-0.000237,0.002051,-0.006541,0.007370,-0.004695,-0.000591,-0.000877,0.005270,0.001961,-0.001481,0.020731,0.004629,0.015264,0.025689,0.096450,0.156145,0.135427,0.046071,0.003212,0.068997,0.037140,0.038636,0.017805,0
2021-09-09,-0.002596,0.026791,-0.004983,-0.001137,0.000710,0.001123,-0.000237,0.002051,-0.006541,0.007370,-0.004695,-0.000591,-0.000877,0.005270,0.001961,0.043176,0.020731,0.004629,0.015264,0.025689,0.096450,0.156145,0.135427,0.046071,0.003212,0.068997,0.037140,0.038636,0
2021-09-10,0.005799,0.094237,-0.002596,-0.004983,-0.001137,0.000710,0.001123,-0.000237,0.002051,-0.006541,0.007370,-0.004695,-0.000591,-0.000877,0.005270,0.026791,0.043176,0.020731,0.004629,0.015264,0.025689,0.096450,0.156145,0.135427,0.046071,0.003212,0.068997,0.037140,0


#Classification

In [ ]:
# Creating a copy of the initial datagrame to make various transformations 
classification_df = pd.DataFrame(final_data.copy())

In [ ]:
classification_df = classification_df.dropna()
classification_df

,CHF/USD Log Difference,CHF/USD_RealVol,CHF/USD Log Difference_lag_1,CHF/USD Log Difference_lag_2,CHF/USD Log Difference_lag_3,CHF/USD Log Difference_lag_4,CHF/USD Log Difference_lag_5,CHF/USD Log Difference_lag_6,CHF/USD Log Difference_lag_7,CHF/USD Log Difference_lag_8,CHF/USD Log Difference_lag_9,CHF/USD Log Difference_lag_10,CHF/USD Log Difference_lag_11,CHF/USD Log Difference_lag_12,CHF/USD Log Difference_lag_13,CHF/USD_RealVol_lag_1,CHF/USD_RealVol_lag_2,CHF/USD_RealVol_lag_3,CHF/USD_RealVol_lag_4,CHF/USD_RealVol_lag_5,CHF/USD_RealVol_lag_6,CHF/USD_RealVol_lag_7,CHF/USD_RealVol_lag_8,CHF/USD_RealVol_lag_9,CHF/USD_RealVol_lag_10,CHF/USD_RealVol_lag_11,CHF/USD_RealVol_lag_12,CHF/USD_RealVol_lag_13,HighVolMoment_CHF/USD
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2003-10-08,0.002515,0.016099,0.003949,0.013026,-0.014013,-0.002203,0.001748,0.004619,0.014178,0.001638,0.005414,0.004943,-0.004352,0.010370,0.013731,0.101894,0.303516,0.132569,0.044348,0.032229,0.107295,0.140761,0.042383,0.005283,0.104335,0.165254,0.037731,0.195514,0
2003-10-09,-0.005326,0.088012,0.002515,0.003949,0.013026,-0.014013,-0.002203,0.001748,0.004619,0.014178,0.001638,0.005414,0.004943,-0.004352,0.010370,0.016099,0.101894,0.303516,0.132569,0.044348,0.032229,0.107295,0.140761,0.042383,0.005283,0.104335,0.165254,0.037731,0
2003-10-10,0.003573,0.099890,-0.005326,0.002515,0.003949,0.013026,-0.014013,-0.002203,0.001748,0.004619,0.014178,0.001638,0.005414,0.004943,-0.004352,0.088012,0.016099,0.101894,0.303516,0.132569,0.044348,0.032229,0.107295,0.140761,0.042383,0.005283,0.104335,0.165254,0
2003-10-13,-0.007285,0.121876,0.003573,-0.005326,0.002515,0.003949,0.013026,-0.014013,-0.002203,0.001748,0.004619,0.014178,0.001638,0.005414,0.004943,0.099890,0.088012,0.016099,0.101894,0.303516,0.132569,0.044348,0.032229,0.107295,0.140761,0.042383,0.005283,0.104335,1
2003-10-14,0.004850,0.136204,-0.007285,0.003573,-0.005326,0.002515,0.003949,0.013026,-0.014013,-0.002203,0.001748,0.004619,0.014178,0.001638,0.005414,0.121876,0.099890,0.088012,0.016099,0.101894,0.303516,0.132569,0.044348,0.032229,0.107295,0.140761,0.042383,0.005283,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-09-08,-0.004983,0.043176,-0.001137,0.000710,0.001123,-0.000237,0.002051,-0.006541,0.007370,-0.004695,-0.000591,-0.000877,0.005270,0.001961,-0.001481,0.020731,0.004629,0.015264,0.025689,0.096450,0.156145,0.135427,0.046071,0.003212,0.068997,0.037140,0.038636,0.017805,0
2021-09-09,-0.002596,0.026791,-0.004983,-0.001137,0.000710,0.001123,-0.000237,0.002051,-0.006541,0.007370,-0.004695,-0.000591,-0.000877,0.005270,0.001961,0.043176,0.020731,0.004629,0.015264,0.025689,0.096450,0.156145,0.135427,0.046071,0.003212,0.068997,0.037140,0.038636,0
2021-09-10,0.005799,0.094237,-0.002596,-0.004983,-0.001137,0.000710,0.001123,-0.000237,0.002051,-0.006541,0.007370,-0.004695,-0.000591,-0.000877,0.005270,0.026791,0.043176,0.020731,0.004629,0.015264,0.025689,0.096450,0.156145,0.135427,0.046071,0.003212,0.068997,0.037140,0


In [ ]:
y = classification_df['HighVolMoment_CHF/USD']
X = classification_df.drop(['CHF/USD Log Difference', 'CHF/USD_RealVol', 'HighVolMoment_CHF/USD'], axis=1)

#Train-Test Split 

In [ ]:
# for time-series cross-validation set 5 folds 
tscv = TimeSeriesSplit(n_splits=5)

In [ ]:
def timeseries_train_test_split(X, y, test_size):
    """
        Perform train-test split with respect to time series structure
    """
    
    # get the index after which test set starts
    test_index = int(len(X)*(1-test_size))
    
    X_train = X.iloc[:test_index]
    y_train = y.iloc[:test_index]
    X_test = X.iloc[test_index:]
    y_test = y.iloc[test_index:]
    
    return X_train, X_test, y_train, y_test

In [ ]:
# reserve 30% of data for testing
X_train, X_test, y_train, y_test = timeseries_train_test_split(X, y, test_size=0.3)

#Testing Models

##Benchmark

##Model 1: Logistic Regression


In [ ]:
from sklearn.linear_model import LogisticRegression
LR1 = LogisticRegression(random_state=42, max_iter=10000).fit(X_train, y_train)

In [ ]:
prob_predictions1 = LR1.predict_proba(X_test)
pred1 = prob_predictions1[:,1]

In [ ]:
fig1 = go.Figure()

fig1.add_trace(go.Scatter(
    name="Raw Data",
    mode="markers+lines", x=X_test.index, y=y_test
))
fig1.add_trace(go.Scatter(
    name="LR Predicted",
    mode="markers+lines", x=X_test.index, y=pred1
))

fig1.show()

In [ ]:
prediction1 = LR1.predict(X_test)

In [ ]:
#Number Error Metrics

print(f"Recall Score: {recall_score(y_test, prediction1)}")
print(f"Accuracy Score: {accuracy_score(y_test, prediction1)}")
print(f"Precision Score: {precision_score(y_test, prediction1)}")
print(f"F1 Score: {f1_score(y_test, prediction1)}")

#Testing Probability Measure as EWS

print(f"RMSE of Probability Score: {mean_squared_error(y_test, pred1, squared=False)}")

Recall Score: 0.00625
Accuracy Score: 0.8853276353276354
Precision Score: 0.3333333333333333
F1 Score: 0.01226993865030675
RMSE of Probability Score: 0.3360082558491576


##Model 2: SGD Classifier

In [ ]:
from sklearn.linear_model import SGDClassifier
from sklearn.calibration import CalibratedClassifierCV
SGD2 = CalibratedClassifierCV(SGDClassifier(random_state=42, max_iter=10000)).fit(X_train, y_train)

In [ ]:
prob_predictions2 = SGD2.predict_proba(X_test)
pred2 = prob_predictions2[:,1]

In [ ]:
fig1 = go.Figure()

fig1.add_trace(go.Scatter(
    name="Raw Data",
    mode="markers+lines", x=X_test.index, y=y_test
))
fig1.add_trace(go.Scatter(
    name="SGD2 Predicted",
    mode="markers+lines", x=X_test.index, y=pred2
))

fig1.show()

In [ ]:
prediction2 = SGD2.predict(X_test)

In [ ]:
#Number Error Metrics

print(f"Recall Score: {recall_score(y_test, prediction2)}")
print(f"Accuracy Score: {accuracy_score(y_test, prediction2)}")
print(f"Precision Score: {precision_score(y_test, prediction2)}")
print(f"F1 Score: {f1_score(y_test, prediction2)}")

#Testing Probability Measure as EWS

print(f"RMSE of Probability Score: {mean_squared_error(y_test, pred2, squared=False)}")

Recall Score: 0.03125
Accuracy Score: 0.8874643874643875
Precision Score: 0.625
F1 Score: 0.05952380952380952
RMSE of Probability Score: 0.33546146983927855


##Model 3: Passive Aggressive Classifier 

In [ ]:
from sklearn.linear_model import PassiveAggressiveClassifier
PAC3 = CalibratedClassifierCV(PassiveAggressiveClassifier(max_iter=10000, random_state=42)).fit(X_train, y_train)

In [ ]:
prob_predictions3 = PAC3.predict_proba(X_test)
pred3 = prob_predictions3[:,1]

In [ ]:
fig1 = go.Figure()

fig1.add_trace(go.Scatter(
    name="Raw Data",
    mode="markers+lines", x=X_test.index, y=y_test
))
fig1.add_trace(go.Scatter(
    name="PAC3 Predicted",
    mode="markers+lines", x=X_test.index, y=pred3
))

fig1.show()

In [ ]:
prediction3 = PAC3.predict(X_test)

In [ ]:
#Number Error Metrics

print(f"Recall Score: {recall_score(y_test, prediction3)}")
print(f"Accuracy Score: {accuracy_score(y_test, prediction3)}")
print(f"Precision Score: {precision_score(y_test, prediction3)}")
print(f"F1 Score: {f1_score(y_test, prediction3)}")

#Testing Probability Measure as EWS

print(f"RMSE of Probability Score: {mean_squared_error(y_test, pred3, squared=False)}")

Recall Score: 0.025
Accuracy Score: 0.8867521367521367
Precision Score: 0.5714285714285714
F1 Score: 0.04790419161676646
RMSE of Probability Score: 0.3347493433259656


##Model 4: Perceptron

In [ ]:
from sklearn.linear_model import Perceptron
Per4 = CalibratedClassifierCV(Perceptron(tol=1e-3, random_state=42)).fit(X_train, y_train)

In [ ]:
prob_predictions4 = Per4.predict_proba(X_test)
pred4 = prob_predictions4[:,1]

In [ ]:
fig1 = go.Figure()

fig1.add_trace(go.Scatter(
    name="Raw Data",
    mode="markers+lines", x=X_test.index, y=y_test
))
fig1.add_trace(go.Scatter(
    name="Per4 Predicted",
    mode="markers+lines", x=X_test.index, y=pred4
))

fig1.show()

In [ ]:
prediction4 = Per4.predict(X_test)

In [ ]:
#Number Error Metrics

print(f"Recall Score: {recall_score(y_test, prediction4)}")
print(f"Accuracy Score: {accuracy_score(y_test, prediction4)}")
print(f"Precision Score: {precision_score(y_test, prediction4)}")
print(f"F1 Score: {f1_score(y_test, prediction4)}")

#Testing Probability Measure as EWS

print(f"RMSE of Probability Score: {mean_squared_error(y_test, pred4, squared=False)}")

Recall Score: 0.00625
Accuracy Score: 0.8853276353276354
Precision Score: 0.3333333333333333
F1 Score: 0.01226993865030675
RMSE of Probability Score: 0.3475750821399242


##Model 5: SVC

In [ ]:
from sklearn.svm import SVC
SVC5 = SVC(probability=True).fit(X_train, y_train)

In [ ]:
prob_predictions5 = SVC5.predict_proba(X_test)
pred5 = prob_predictions5[:,1]

In [ ]:
fig1 = go.Figure()

fig1.add_trace(go.Scatter(
    name="Raw Data",
    mode="markers+lines", x=X_test.index, y=y_test
))
fig1.add_trace(go.Scatter(
    name="SVC5 Predicted",
    mode="markers+lines", x=X_test.index, y=pred5
))

fig1.show()

In [ ]:
prediction5 = SVC5.predict(X_test)

In [ ]:
#Number Error Metrics

print(f"Recall Score: {recall_score(y_test, prediction5)}")
print(f"Accuracy Score: {accuracy_score(y_test, prediction5)}")
print(f"Precision Score: {precision_score(y_test, prediction5)}")
print(f"F1 Score: {f1_score(y_test, prediction5)}")

#Testing Probability Measure as EWS

print(f"RMSE of Probability Score: {mean_squared_error(y_test, pred5, squared=False)}")

Recall Score: 0.01875
Accuracy Score: 0.886039886039886
Precision Score: 0.5
F1 Score: 0.03614457831325301
RMSE of Probability Score: 0.3441079734269821


##Model 6: NuSVC

In [ ]:
from sklearn.svm import NuSVC
NuSVC6 = NuSVC(probability=True).fit(X_train, y_train)

In [ ]:
prob_predictions6 = NuSVC6.predict_proba(X_test)
pred6 = prob_predictions6[:,1]

In [ ]:
fig1 = go.Figure()

fig1.add_trace(go.Scatter(
    name="Raw Data",
    mode="markers+lines", x=X_test.index, y=y_test
))
fig1.add_trace(go.Scatter(
    name="SVC5 Predicted",
    mode="markers+lines", x=X_test.index, y=pred6
))

fig1.show()

In [ ]:
prediction6 = NuSVC6.predict(X_test)

In [ ]:
#Number Error Metrics

print(f"Recall Score: {recall_score(y_test, prediction6)}")
print(f"Accuracy Score: {accuracy_score(y_test, prediction6)}")
print(f"Precision Score: {precision_score(y_test, prediction6)}")
print(f"F1 Score: {f1_score(y_test, prediction6)}")

#Testing Probability Measure as EWS

print(f"RMSE of Probability Score: {mean_squared_error(y_test, pred6, squared=False)}")

Recall Score: 0.0875
Accuracy Score: 0.886039886039886
Precision Score: 0.5
F1 Score: 0.14893617021276595
RMSE of Probability Score: 0.34726571122465355


##Model 7: Linear SVC

In [ ]:
from sklearn.svm import LinearSVC
LinSVC7 = CalibratedClassifierCV(LinearSVC(random_state = 42)).fit(X_train, y_train)

In [ ]:
prob_predictions7 = LinSVC7.predict_proba(X_test)
pred7 = prob_predictions7[:,1]

In [ ]:
fig1 = go.Figure()

fig1.add_trace(go.Scatter(
    name="Raw Data",
    mode="markers+lines", x=X_test.index, y=y_test
))
fig1.add_trace(go.Scatter(
    name="LinSVC7 Predicted",
    mode="markers+lines", x=X_test.index, y=pred7
))

fig1.show()

In [ ]:
prediction7 = LinSVC7.predict(X_test)

In [ ]:
#Number Error Metrics

print(f"Recall Score: {recall_score(y_test, prediction7)}")
print(f"Accuracy Score: {accuracy_score(y_test, prediction7)}")
print(f"Precision Score: {precision_score(y_test, prediction7)}")
print(f"F1 Score: {f1_score(y_test, prediction7)}")

#Testing Probability Measure as EWS

print(f"RMSE of Probability Score: {mean_squared_error(y_test, pred7, squared=False)}")

Recall Score: 0.03125
Accuracy Score: 0.8874643874643875
Precision Score: 0.625
F1 Score: 0.05952380952380952
RMSE of Probability Score: 0.33341186604075296


##Model 8: K Neighbors

In [ ]:
from sklearn import neighbors
KNN8 = neighbors.KNeighborsClassifier().fit(X_train, y_train)

In [ ]:
prob_predictions8 = KNN8.predict_proba(X_test)
pred8 = prob_predictions8[:,1]

In [ ]:
fig1 = go.Figure()

fig1.add_trace(go.Scatter(
    name="Raw Data",
    mode="markers+lines", x=X_test.index, y=y_test
))
fig1.add_trace(go.Scatter(
    name="SVR8 Predicted",
    mode="markers+lines", x=X_test.index, y=pred8
))

fig1.show()

In [ ]:
prediction8 = KNN8.predict(X_test)

In [ ]:
#Number Error Metrics

print(f"Recall Score: {recall_score(y_test, prediction8)}")
print(f"Accuracy Score: {accuracy_score(y_test, prediction8)}")
print(f"Precision Score: {precision_score(y_test, prediction8)}")
print(f"F1 Score: {f1_score(y_test, prediction8)}")

#Testing Probability Measure as EWS

print(f"RMSE of Probability Score: {mean_squared_error(y_test, pred8, squared=False)}")

Recall Score: 0.11875
Accuracy Score: 0.8582621082621082
Precision Score: 0.24675324675324675
F1 Score: 0.16033755274261605
RMSE of Probability Score: 0.3592886292221326


##Model 9: Gaussian Process Classification

In [ ]:
from sklearn.gaussian_process.kernels import ConstantKernel, RBF
from sklearn.gaussian_process import GaussianProcessClassifier

#kernel = ConstantKernel(constant_value=1.0, constant_value_bounds=(0.0, 10.0)) * RBF(length_scale=0.5, length_scale_bounds=(0.0, 10.0)) + RBF(length_scale=2.0, length_scale_bounds=(0.0, 10.0))
kernel = ConstantKernel() * RBF(length_scale=0.5, length_scale_bounds=(0.0, 10.0))
GPC9 = GaussianProcessClassifier(kernel=kernel, random_state=42).fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/gaussian_process/kernels.py:335: RuntimeWarning:

divide by zero encountered in log



In [ ]:
prob_predictions9 = GPC9.predict_proba(X_test)
pred9 = prob_predictions9[:,1]

In [ ]:
fig1 = go.Figure()

fig1.add_trace(go.Scatter(
    name="Raw Data",
    mode="markers+lines", x=X_test.index, y=y_test
))
fig1.add_trace(go.Scatter(
    name="GPC9 Predicted",
    mode="markers+lines", x=X_test.index, y=pred9
))

fig1.show()

In [ ]:
prediction9 = GPC9.predict(X_test)

In [ ]:
#Number Error Metrics

print(f"Recall Score: {recall_score(y_test, prediction9)}")
print(f"Accuracy Score: {accuracy_score(y_test, prediction9)}")
print(f"Precision Score: {precision_score(y_test, prediction9)}")
print(f"F1 Score: {f1_score(y_test, prediction9)}")

#Testing Probability Measure as EWS

print(f"RMSE of Probability Score: {mean_squared_error(y_test, pred9, squared=False)}")

Recall Score: 0.04375
Accuracy Score: 0.8831908831908832
Precision Score: 0.3888888888888889
F1 Score: 0.07865168539325841
RMSE of Probability Score: 0.32067185474191007


#Model 10: Naive Bayes

In [ ]:
from sklearn.naive_bayes import GaussianNB
GNB10 = GaussianNB().fit(X_train, y_train)

In [ ]:
prob_predictions10 = GNB10.predict_proba(X_test)
pred10 = prob_predictions10[:,1]

In [ ]:
fig1 = go.Figure()

fig1.add_trace(go.Scatter(
    name="Raw Data",
    mode="markers+lines", x=X_test.index, y=y_test
))
fig1.add_trace(go.Scatter(
    name="GPC9 Predicted",
    mode="markers+lines", x=X_test.index, y=pred10
))

fig1.show()

In [ ]:
prediction10 = GNB10.predict(X_test)

In [ ]:
#Number Error Metrics

print(f"Recall Score: {recall_score(y_test, prediction10)}")
print(f"Accuracy Score: {accuracy_score(y_test, prediction10)}")
print(f"Precision Score: {precision_score(y_test, prediction10)}")
print(f"F1 Score: {f1_score(y_test, prediction10)}")

#Testing Probability Measure as EWS

print(f"RMSE of Probability Score: {mean_squared_error(y_test, pred10, squared=False)}")

Recall Score: 0.025
Accuracy Score: 0.8817663817663818
Precision Score: 0.2857142857142857
F1 Score: 0.04597701149425287
RMSE of Probability Score: 0.3377499766974664


#Model 11: Decision Tree Classifier

In [ ]:
from sklearn.tree import DecisionTreeClassifier
DTC11 = DecisionTreeClassifier(random_state=42).fit(X_train, y_train)

In [ ]:
prob_predictions11 = DTC11.predict_proba(X_test)
pred11 = prob_predictions11[:,1]

In [ ]:
fig1 = go.Figure()

fig1.add_trace(go.Scatter(
    name="Raw Data",
    mode="markers+lines", x=X_test.index, y=y_test
))
fig1.add_trace(go.Scatter(
    name="DTC11 Predicted",
    mode="markers+lines", x=X_test.index, y=pred11
))

fig1.show()

In [ ]:
prediction11 = DTC11.predict(X_test)

In [ ]:
#Number Error Metrics

print(f"Recall Score: {recall_score(y_test, prediction11)}")
print(f"Accuracy Score: {accuracy_score(y_test, prediction11)}")
print(f"Precision Score: {precision_score(y_test, prediction11)}")
print(f"F1 Score: {f1_score(y_test, prediction11)}")

#Testing Probability Measure as EWS

print(f"RMSE of Probability Score: {mean_squared_error(y_test, pred11, squared=False)}")

Recall Score: 0.35
Accuracy Score: 0.7685185185185185
Precision Score: 0.20216606498194944
F1 Score: 0.2562929061784897
RMSE of Probability Score: 0.4811252243246881


##Model 12: Baggin KN Classifier

In [ ]:
from sklearn.ensemble import BaggingClassifier
from sklearn.neighbors import KNeighborsClassifier
BAGGING12 = BaggingClassifier(KNeighborsClassifier(), max_samples=0.5, max_features=0.5).fit(X_train, y_train)

In [ ]:
prob_predictions12 = BAGGING12.predict_proba(X_test)
pred12 = prob_predictions12[:,1]

In [ ]:
fig1 = go.Figure()

fig1.add_trace(go.Scatter(
    name="Raw Data",
    mode="markers+lines", x=X_test.index, y=y_test
))
fig1.add_trace(go.Scatter(
    name="BAGGING12 Predicted",
    mode="markers+lines", x=X_test.index, y=pred12
))

fig1.show()

In [ ]:
prediction12 = BAGGING12.predict(X_test)

In [ ]:
#Number Error Metrics

print(f"Recall Score: {recall_score(y_test, prediction11)}")
print(f"Accuracy Score: {accuracy_score(y_test, prediction11)}")
print(f"Precision Score: {precision_score(y_test, prediction11)}")
print(f"F1 Score: {f1_score(y_test, prediction11)}")

#Testing Probability Measure as EWS

print(f"RMSE of Probability Score: {mean_squared_error(y_test, pred11, squared=False)}")

Recall Score: 0.35
Accuracy Score: 0.7685185185185185
Precision Score: 0.20216606498194944
F1 Score: 0.2562929061784897
RMSE of Probability Score: 0.4811252243246881


##Model 13: Random Forest Classifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier
RF13 = RandomForestClassifier(n_estimators=1000).fit(X_train, y_train)

In [ ]:
prob_predictions13 = RF13.predict_proba(X_test)
pred13 = prob_predictions13[:,1]

In [ ]:
fig1 = go.Figure()

fig1.add_trace(go.Scatter(
    name="Raw Data",
    mode="markers+lines", x=X_test.index, y=y_test
))
fig1.add_trace(go.Scatter(
    name="RF13 Predicted",
    mode="markers+lines", x=X_test.index, y=pred13
))

fig1.show()

In [ ]:
prediction13 = RF13.predict(X_test)

In [ ]:
#Number Error Metrics

print(f"Recall Score: {recall_score(y_test, prediction13)}")
print(f"Accuracy Score: {accuracy_score(y_test, prediction13)}")
print(f"Precision Score: {precision_score(y_test, prediction13)}")
print(f"F1 Score: {f1_score(y_test, prediction13)}")

#Testing Probability Measure as EWS

print(f"RMSE of Probability Score: {mean_squared_error(y_test, pred13, squared=False)}")

Recall Score: 0.19375
Accuracy Score: 0.8881766381766382
Precision Score: 0.5254237288135594
F1 Score: 0.28310502283105027
RMSE of Probability Score: 0.3087477014690039


##Model 14: Extreme Randomized Trees

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier
ETC14 = ExtraTreesClassifier(random_state=42).fit(X_train, y_train)

In [ ]:
prob_predictions14 = ETC14.predict_proba(X_test)
pred14 = prob_predictions14[:,1]

In [ ]:
fig1 = go.Figure()

fig1.add_trace(go.Scatter(
    name="Raw Data",
    mode="markers+lines", x=X_test.index, y=y_test
))
fig1.add_trace(go.Scatter(
    name="ETC14 Predicted",
    mode="markers+lines", x=X_test.index, y=pred14
))

fig1.show()

In [ ]:
prediction14 = ETC14.predict(X_test)

In [ ]:
#Number Error Metrics

print(f"Recall Score: {recall_score(y_test, prediction14)}")
print(f"Accuracy Score: {accuracy_score(y_test, prediction14)}")
print(f"Precision Score: {precision_score(y_test, prediction14)}")
print(f"F1 Score: {f1_score(y_test, prediction14)}")

#Testing Probability Measure as EWS

print(f"RMSE of Probability Score: {mean_squared_error(y_test, pred14, squared=False)}")

Recall Score: 0.09375
Accuracy Score: 0.8867521367521367
Precision Score: 0.5172413793103449
F1 Score: 0.15873015873015875
RMSE of Probability Score: 0.31111736867943757


#Model 15: AdaBoost CLassifier

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
AB15 = AdaBoostClassifier(n_estimators=100).fit(X_train, y_train)

In [ ]:
prob_predictions15 = AB15.predict_proba(X_test)
pred15 = prob_predictions15[:,1]

In [ ]:
fig1 = go.Figure()

fig1.add_trace(go.Scatter(
    name="Raw Data",
    mode="markers+lines", x=X_test.index, y=y_test
))
fig1.add_trace(go.Scatter(
    name="AB15 Predicted",
    mode="markers+lines", x=X_test.index, y=pred15
))

fig1.show()

In [ ]:
prediction15 = AB15.predict(X_test)

In [ ]:
#Number Error Metrics

print(f"Recall Score: {recall_score(y_test, prediction15)}")
print(f"Accuracy Score: {accuracy_score(y_test, prediction15)}")
print(f"Precision Score: {precision_score(y_test, prediction15)}")
print(f"F1 Score: {f1_score(y_test, prediction15)}")

#Testing Probability Measure as EWS

print(f"RMSE of Probability Score: {mean_squared_error(y_test, pred15, squared=False)}")

Recall Score: 0.1875
Accuracy Score: 0.8846153846153846
Precision Score: 0.4838709677419355
F1 Score: 0.2702702702702703
RMSE of Probability Score: 0.49362864981673715


##Mode 16: Gradient Tree Boosting

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
GTB16 = GradientBoostingClassifier(random_state = 42).fit(X_train, y_train)

In [ ]:
prob_predictions16 = GTB16.predict_proba(X_test)
pred16 = prob_predictions16[:,1]

In [ ]:
fig1 = go.Figure()

fig1.add_trace(go.Scatter(
    name="Raw Data",
    mode="markers+lines", x=X_test.index, y=y_test
))
fig1.add_trace(go.Scatter(
    name="GTB16 Predicted",
    mode="markers+lines", x=X_test.index, y=pred16
))

fig1.show()

In [ ]:
prediction16 = GTB16.predict(X_test)

In [ ]:
#Number Error Metrics

print(f"Recall Score: {recall_score(y_test, prediction16)}")
print(f"Accuracy Score: {accuracy_score(y_test, prediction16)}")
print(f"Precision Score: {precision_score(y_test, prediction16)}")
print(f"F1 Score: {f1_score(y_test, prediction16)}")

#Testing Probability Measure as EWS

print(f"RMSE of Probability Score: {mean_squared_error(y_test, pred16, squared=False)}")

Recall Score: 0.175
Accuracy Score: 0.8846153846153846
Precision Score: 0.4827586206896552
F1 Score: 0.25688073394495414
RMSE of Probability Score: 0.30625482271624066


##Model 17: Voting CLassifier

In [ ]:
from sklearn.ensemble import VotingClassifier

clf1 = LogisticRegression(random_state=1, max_iter=1000)
clf2 = RandomForestClassifier(n_estimators=50, random_state=1)
clf3 = GaussianNB()

eclf = VotingClassifier(estimators=[('lr', clf1), ('rf', clf2), ('gnb', clf3)], voting='hard')

for clf, label in zip([clf1, clf2, clf3, eclf], ['Logistic Regression', 'Random Forest', 'naive Bayes', 'Ensemble']):
  scores = cross_val_score(clf, X_train, y_train, scoring='accuracy', cv=5)
  print("Accuracy: %0.2f (+/- %0.2f) [%s]" % (scores.mean(), scores.std(), label))

Accuracy: 0.69 (+/- 0.01) [Logistic Regression]
Accuracy: 0.74 (+/- 0.02) [Random Forest]
Accuracy: 0.70 (+/- 0.02) [naive Bayes]
Accuracy: 0.71 (+/- 0.02) [Ensemble]


##Model 18: XGBoost

In [ ]:
XGB18 = XGBClassifier(random_state=42).fit(X_train, y_train)

In [ ]:
prob_predictions18 = XGB18.predict_proba(X_test)
pred18 = prob_predictions18[:,1]

In [ ]:
fig1 = go.Figure()

fig1.add_trace(go.Scatter(
    name="Raw Data",
    mode="markers+lines", x=X_test.index, y=y_test
))
fig1.add_trace(go.Scatter(
    name="XGB18 Predicted",
    mode="markers+lines", x=X_test.index, y=pred18
))

fig1.show()

In [ ]:
prediction18 = XGB18.predict(X_test)

In [ ]:
#Number Error Metrics

print(f"Recall Score: {recall_score(y_test, prediction18)}")
print(f"Accuracy Score: {accuracy_score(y_test, prediction18)}")
print(f"Precision Score: {precision_score(y_test, prediction18)}")
print(f"F1 Score: {f1_score(y_test, prediction18)}")

#Testing Probability Measure as EWS

print(f"RMSE of Probability Score: {mean_squared_error(y_test, pred18, squared=False)}")

Recall Score: 0.1875
Accuracy Score: 0.8867521367521367
Precision Score: 0.5084745762711864
F1 Score: 0.273972602739726
RMSE of Probability Score: 0.30653785188123833


#Model 19: MLP Classification

In [ ]:
from sklearn.neural_network import MLPClassifier
MLP19 = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(5, 2), random_state=42).fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:549: ConvergenceWarning:

lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html



In [ ]:
prob_predictions19 = MLP19.predict_proba(X_test)
pred19 = prob_predictions19[:,1]

In [ ]:
fig1 = go.Figure()

fig1.add_trace(go.Scatter(
    name="Raw Data",
    mode="markers+lines", x=X_test.index, y=y_test
))
fig1.add_trace(go.Scatter(
    name="MLP19 Predicted",
    mode="markers+lines", x=X_test.index, y=pred19
))

fig1.show()

In [ ]:
prediction19 = MLP19.predict(X_test)

In [ ]:
#Number Error Metrics

print(f"Recall Score: {recall_score(y_test, prediction19)}")
print(f"Accuracy Score: {accuracy_score(y_test, prediction19)}")
print(f"Precision Score: {precision_score(y_test, prediction19)}")
print(f"F1 Score: {f1_score(y_test, prediction19)}")

#Testing Probability Measure as EWS

print(f"RMSE of Probability Score: {mean_squared_error(y_test, pred19, squared=False)}")

Recall Score: 0.05
Accuracy Score: 0.8867521367521367
Precision Score: 0.5333333333333333
F1 Score: 0.09142857142857143
RMSE of Probability Score: 0.3201924139276055


##Model 20: MLPClassification Large

In [ ]:
MLP20 = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(10, 10), random_state=42).fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:549: ConvergenceWarning:

lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html



In [ ]:
prob_predictions20 = MLP20.predict_proba(X_test)
pred20 = prob_predictions20[:,1]

In [ ]:
fig1 = go.Figure()

fig1.add_trace(go.Scatter(
    name="Raw Data",
    mode="markers+lines", x=X_test.index, y=y_test
))
fig1.add_trace(go.Scatter(
    name="MLP20 Predicted",
    mode="markers+lines", x=X_test.index, y=pred20
))

fig1.show()

In [ ]:
prediction20 = MLP20.predict(X_test)

In [ ]:
#Number Error Metrics

print(f"Recall Score: {recall_score(y_test, prediction20)}")
print(f"Accuracy Score: {accuracy_score(y_test, prediction20)}")
print(f"Precision Score: {precision_score(y_test, prediction20)}")
print(f"F1 Score: {f1_score(y_test, prediction20)}")

#Testing Probability Measure as EWS

print(f"RMSE of Probability Score: {mean_squared_error(y_test, pred20, squared=False)}")

Recall Score: 0.1125
Accuracy Score: 0.886039886039886
Precision Score: 0.5
F1 Score: 0.18367346938775508
RMSE of Probability Score: 0.3224240151512054
